In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import torch
import _pickle as pickle
import scipy.stats as stats
import scipy
from plotly.subplots import make_subplots
import plotly.io as pio
import sys, os
from statsmodels.stats.multitest import multipletests as multi_tests

# Global Variables

In [2]:
RESULT_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results"
REF_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REF"
TRANS_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/TRANS"
REDUC_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REDUC"
METRICS = ["recon_error","adv_error", "rot_error", "delta_rot"]
METRICS_RENAME = ["Re. error", "Adv. error", "Rot. error", "RMS"]
SUFFIX = {"Raw":"_RAW", "Frozen":"_F", "Trained":"_T"}


# Functions

In [89]:
def pJoin(p1, p2):
    return os.path.join(p1,p2)

def load(file_path:str):
    with bz2.BZ2File(file_path, "rb") as f:
        obj = pickle.load(f)
    return obj
def save(file:object, file_path:str):
    with bz2.BZ2File(file_path, "w") as f:
        pickle.dump(file, f)
def mean_confidence_interval(data, confidence=0.95):
    data = np.nan_to_num(data, 0)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def plot_bar(results:dict, title="", x_axis_name=None, y_axis_name=None, 
                    width=500, height=300, vertical=False,
                    add_hline=False, annotation_text="baseline", annotation_position="top"):

    if not vertical:
        fig = ex.bar(results, x="Models", y="Percentage", color="Metrics", opacity=0.8, text="Percentage", template="seaborn")
    else:
        fig = ex.bar(results, y="Models", x="Percentage", color="Metrics", opacity=0.8, text="Percentage", template="seaborn")

    if add_hline:
        if not vertical:
            fig.add_hline(y=400, x0=-0.5, x1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 
        else:
            fig.add_vline(x=400, y0=-0.5, y1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 

    fig.update_layout(
    #    title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)

    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text=y_axis_name
        )
    return fig

def save_plot(plot:object, file_path="figure.svg"):
    plot.write_image(file_path)

def calc_diff(*results, multiplier=100, inverse=False):
    if len(results) < 2:
        raise ValueError("Need at least 2 arrays for comparison")
    if (len(results[0])) < 2:
        raise ValueError("The result array should contain results from each rig")
    if not inverse:
        comp_fn = lambda base,target: np.asarray([1 / (y / x) * multiplier for x,y in zip(base, target)], np.float64) 
    else:
        comp_fn = lambda base,target: np.asarray([(y / x) * multiplier for x,y in zip(base, target)], np.float64) 

    base = results[0]
    comparison = [[],[]]
    for target in results[1:]:
        comp_avg_over_rig = np.round(np.mean([comp_fn(b,t) for b,t in zip(base, target)]), decimals=2)
        pval_avg_over_rig = np.max(multi_tests([stats.kruskal(t,b)[1] for b,t in zip(base, target)])[1])
        comparison[0].append(comp_avg_over_rig)
        comparison[1].append(pval_avg_over_rig)
    return comparison

def calc_pvals(*results):
    null_hypothesis = {}
    for metric in METRICS:
        p_vals = [stats.kruskal(*[m[metric][i] for m in results])[1] for i in range(len(results[0][metric]))]
        adjusted_pvals = np.max(multi_tests(p_vals)[1])
        null_hypothesis[metric] = np.round(adjusted_pvals,3)
    return null_hypothesis


# Generation performance

## MoE vs LSTM

In [4]:
file_names = [] 
for _, _, names in os.walk(REF_SAVE_PATH):
    file_names = names
print(file_names)


['AE+LSTM.pbz2', 'AE+MoE.pbz2', 'DEC-CAT+MoE.pbz2', 'DEC-IN+MoE.pbz2', 'RBF-CAT+LSTM.pbz2', 'RBF-CAT+MoE.pbz2', 'RBF-IN+LSTM.pbz2', 'RBF-IN+MoE.pbz2', 'VAE-CAT+LSTM.pbz2', 'VAE-CAT+MoE.pbz2', 'VAE-IN+LSTM.pbz2', 'VAE-IN+MoE.pbz2']


In [92]:
comp_df = {"AE":{}, "RBF-IN":{}, "RBF-CAT":{}, "VAE-IN":{}, "VAE-CAT":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"+LSTM.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [93]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2")) for k in keys]
for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["AE_vs_RBF-CAT_vs_VAE-CAT"] = calc_pvals(res[0],res[2],res[4])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/ref/MoE_all.csv")

In [17]:
null_hypothesis = {}
MoEs = [load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2")) for k in comp_df.keys()]
for metric in METRICS:
    p_stats = np.mean([
        stats.kruskal(
            MoEs[0][metric][i],
            MoEs[4][metric][i]
            # MoEs[2][metric][i],
            # MoEs[3][metric][i],
            # MoEs[4][metric][i]
            )[1] for i in range(5)
        ])
    null_hypothesis[metric] = p_stats
print(null_hypothesis)

{'recon_error': 0.12346279147061798, 'adv_error': 0.40870249625747157, 'rot_error': 0.09015967834665307, 'delta_rot': 0.01891790721886579}


In [51]:
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of LSTM compared to MoE", 
        width=600, add_hline=True, annotation_text="baseline")
save_plot(fig, "img/ref/MoE_vs_LSTM.svg")
fig.show()

In [60]:
for k in keys:
    for metric in METRICS:
        # comp_df[k][metric]= (comp_df[k][metric][0][0], comp_df[k][metric][1][0])
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1], 3)[0])

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/MoE_vs_LSTM.csv")

                          AE          RBF-IN          RBF-CAT          VAE-IN  \
recon_error     (31.94, 0.0)   (80.8, 0.016)     (30.85, 0.0)    (58.47, 0.0)   
adv_error    (121.17, 0.459)  (112.29, 0.01)  (122.08, 0.853)   (115.84, 0.0)   
rot_error       (75.04, 0.0)  (95.52, 0.169)     (77.88, 0.0)    (94.63, 0.0)   
delta_rot       (58.85, 0.0)    (79.34, 0.0)     (57.48, 0.0)  (148.03, 0.17)   

                     VAE-CAT  
recon_error      (37.1, 0.0)  
adv_error    (123.86, 0.035)  
rot_error       (82.43, 0.0)  
delta_rot       (64.64, 0.0)  


## FC-CAT vs FC-IN

In [95]:
comp_df = {"RBF":{}, "VAE":{}, "DEC":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"-IN+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"-CAT+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])


In [96]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(REF_SAVE_PATH,k+"-CAT+MoE.pbz2")) for k in keys]
for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/ref/FC-CAT_all.csv")

In [69]:
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FC-IN", 
        width=600, add_hline=True, annotation_text="")
save_plot(fig, "img/ref/FC-IN_vs_FC-CAT.svg")
fig.show()

In [70]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/FC-IN_vs_FC-CAT.csv")

                         RBF             VAE              DEC
recon_error    (397.77, 0.0)   (247.53, 0.0)  (172.77, 0.586)
adv_error    (101.55, 0.008)  (98.85, 0.808)  (104.77, 0.127)
rot_error      (140.29, 0.0)   (129.39, 0.0)   (111.71, 0.31)
delta_rot      (143.19, 0.0)   (260.38, 0.0)    (138.36, 0.0)


## FE vs FC-CAT

In [94]:
comp_df = {"RBF-CAT":{}, "VAE-CAT":{}, "DEC-CAT":{}}
f1 = load(pJoin(REF_SAVE_PATH,"AE+MoE.pbz2"))
for k in comp_df.keys():
    f2 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [72]:
file_name = "FE_vs_FC-CAT"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FE", 
        width=600, add_hline=True, annotation_text="")
save_plot(fig, "img/ref/"+file_name+".svg")
fig.show()

In [73]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/"+file_name+".csv")

                     RBF-CAT          VAE-CAT          DEC-CAT
recon_error   (112.49, 0.02)  (100.92, 0.616)      (48.6, 0.0)
adv_error    (100.07, 0.917)  (100.25, 0.935)  (106.26, 0.393)
rot_error    (102.46, 0.941)   (97.24, 0.279)   (82.33, 0.282)
delta_rot      (99.42, 0.97)   (84.77, 0.095)   (91.69, 0.911)


# Transferred quality
## FE Transferred (frozen, trained) vs FE Reference 

In [4]:
file_names = [] 
for _, _, names in os.walk(TRANS_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_RAW.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_RAW.pbz2', 'DEC-CAT+MoE_T.pbz2', 'DEC-IN+MoE_F.pbz2', 'DEC-IN+MoE_RAW.pbz2', 'DEC-IN+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_RAW.pbz2', 'RBF-CAT+MoE_T.pbz2', 'RBF-IN+MoE_F.pbz2', 'RBF-IN+MoE_RAW.pbz2', 'RBF-IN+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_RAW.pbz2', 'VAE-CAT+MoE_T.pbz2', 'VAE-IN+MoE_F.pbz2', 'VAE-IN+MoE_RAW.pbz2', 'VAE-IN+MoE_T.pbz2']


In [98]:
comp_df = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(REF_SAVE_PATH, base+".pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

In [99]:
null_comp_df = {} 
keys = list(comp_df.keys())
res = [load(pJoin(TRANS_SAVE_PATH,base+SUFFIX[k]+".pbz2")) for k in keys]

for k in range(len(keys)):
    for j in range(k+1,len(keys)):
        pvals = calc_pvals(res[k],res[j])      
        null_comp_df[keys[k] + "_vs_" + keys[j]] = pvals

null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/ae_all.csv")

In [101]:
file_name = "FE_RAW_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Transferring quality of FE", 
        width=600, add_hline=True, annotation_text="Reference perf.")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [102]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                         Raw           Frozen          Trained
recon_error     (39.37, 0.0)      (76.8, 0.0)     (67.99, 0.0)
adv_error       (91.32, 0.0)   (98.75, 0.497)    (98.0, 0.161)
rot_error       (72.34, 0.0)     (89.96, 0.0)   (88.76, 0.001)
delta_rot    (116.32, 0.002)  (100.85, 0.315)  (105.56, 0.374)


In [103]:
comp_df = {"Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(TRANS_SAVE_PATH, base+"_RAW.pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [104]:
file_name = "FE_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Performance improvment of FE", 
        width=600, add_hline=True, annotation_text="Cold perf.", annotation_position="bottom")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [105]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                     Frozen          Trained
recon_error   (198.31, 0.0)    (184.09, 0.0)
adv_error     (108.17, 0.0)  (107.91, 0.001)
rot_error     (126.17, 0.0)    (124.48, 0.0)
delta_rot    (92.56, 0.002)   (96.05, 0.209)


## FC
### RBF

In [106]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])


In [112]:
null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/rbf_all.csv")

In [113]:
file_name = "RBF_RAW_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(4)] + ["RBF-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - RBF", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [114]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                        Raw          Frozen         Trained
recon_error    (92.04, 0.0)    (94.97, 0.0)  (103.04, 0.02)
adv_error      (87.76, 0.0)  (96.67, 0.001)  (98.79, 0.146)
rot_error    (92.15, 0.406)  (91.47, 0.513)    (95.0, 0.01)
delta_rot     (168.08, 0.0)  (97.49, 0.984)  (98.61, 0.001)
                         Raw          Frozen          Trained
recon_error     (39.83, 0.0)    (67.86, 0.0)      (63.6, 0.0)
adv_error       (91.54, 0.0)  (97.52, 0.342)    (96.5, 0.377)
rot_error       (71.36, 0.0)    (86.86, 0.0)     (84.66, 0.0)
delta_rot    (107.72, 0.545)   (98.8, 0.878)  (100.62, 0.146)


In [115]:
# Against RAW

comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])


In [116]:
file_name = "RBF_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(4)] + ["RBF-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvment of FC - RBF", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [117]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen          Trained
recon_error  (103.99, 0.927)   (114.8, 0.005)
adv_error    (112.96, 0.034)    (114.04, 0.0)
rot_error     (100.0, 0.674)  (103.87, 0.001)
delta_rot       (62.72, 0.0)     (62.95, 0.0)
                     Frozen         Trained
recon_error   (171.43, 0.0)   (166.63, 0.0)
adv_error     (107.63, 0.0)   (106.47, 0.0)
rot_error     (122.57, 0.0)   (120.62, 0.0)
delta_rot    (93.68, 0.001)  (96.56, 0.407)


### VAE

In [118]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/vae_all.csv")

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                       Raw           Frozen          Trained
recon_error    (63.2, 0.0)     (81.66, 0.0)     (97.45, 0.0)
adv_error     (81.63, 0.0)     (92.35, 0.0)   (94.49, 0.004)
rot_error     (88.35, 0.0)      (90.9, 0.0)    (94.98, 0.79)
delta_rot    (259.67, 0.0)  (127.69, 0.669)  (111.81, 0.133)
                      Raw          Frozen          Trained
recon_error  (48.76, 0.0)    (78.43, 0.0)   (87.41, 0.018)
adv_error    (95.73, 0.0)  (94.79, 0.024)   (99.51, 0.207)
rot_error    (80.34, 0.0)  (91.52, 0.005)   (98.04, 0.151)
delta_rot    (61.05, 0.0)    (84.23, 0.0)  (101.13, 0.081)


In [119]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen          Trained
recon_error    (132.7, 0.01)     (164.4, 0.0)
adv_error      (115.71, 0.0)    (119.37, 0.0)
rot_error    (103.16, 0.852)  (108.55, 0.004)
delta_rot       (55.59, 0.0)     (50.47, 0.0)
                    Frozen        Trained
recon_error  (167.02, 0.0)  (183.63, 0.0)
adv_error     (99.76, 0.0)  (104.43, 0.0)
rot_error    (115.13, 0.0)  (123.69, 0.0)
delta_rot    (147.09, 0.0)  (172.85, 0.0)


In [120]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

null_comp_df = {} 
keys = list(comp_df_in.keys())
m_keys = [base+"-IN+MoE"+SUFFIX[k] for k in keys] + [base+"-CAT+MoE"+SUFFIX[k] for k in keys]
res = [load(pJoin(TRANS_SAVE_PATH,k + ".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")
        k2 = m_keys[j].replace(base,"").replace("-IN","IN").replace("-CAT","CAT").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["IN"] = calc_pvals(res[0],res[1],res[2])
null_comp_df["CAT"] = calc_pvals(res[3],res[4],res[5])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/trans/dec_all.csv")

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                        Raw          Frozen         Trained
recon_error    (91.07, 0.0)    (91.48, 0.0)  (107.1, 0.025)
adv_error      (86.76, 0.0)  (96.27, 0.599)   (98.2, 0.781)
rot_error    (92.55, 0.872)    (91.43, 0.0)  (98.89, 0.107)
delta_rot      (183.5, 0.0)   (105.28, 0.0)  (111.1, 0.992)
                         Raw          Frozen          Trained
recon_error     (79.26, 0.0)  (80.03, 0.069)    (114.51, 0.0)
adv_error     (90.51, 0.014)  (87.04, 0.067)   (95.39, 0.271)
rot_error     (89.79, 0.009)    (83.46, 0.0)     (98.22, 0.0)
delta_rot    (138.35, 0.785)  (89.98, 0.009)  (103.73, 0.993)


In [121]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen        Trained
recon_error  (102.76, 0.183)   (118.7, 0.0)
adv_error      (112.94, 0.0)  (114.53, 0.0)
rot_error     (99.66, 0.003)  (107.57, 0.0)
delta_rot       (64.54, 0.0)   (66.62, 0.0)
                    Frozen          Trained
recon_error   (98.18, 0.0)    (145.41, 0.0)
adv_error      (96.9, 0.0)  (106.07, 0.953)
rot_error    (94.55, 0.65)   (111.2, 0.282)
delta_rot     (66.84, 0.0)     (78.49, 0.0)


## REDUC TRANSFERRING QUALITY

In [23]:
file_names = [] 
for _, _, names in os.walk(REDUC_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_T.pbz2']


In [123]:
comp_df_ae = {"Frozen":{}, "Trained":{}}
comp_df_rbf = {"Frozen":{}, "Trained":{}}
comp_df_vae = {"Frozen":{}, "Trained":{}}
comp_df_dec = {"Frozen":{}, "Trained":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = SUFFIX[k]
        if base == "AE":
            f1 = load(pJoin(REF_SAVE_PATH, base+"+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"+MoE"+suffix+".pbz2"))
        else:
            f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
            
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [128]:
null_comp_df = {}
bases = ["AE", "RBF", "VAE", "DEC"] 
keys = list(comp_df_ae.keys())
m_keys =    ["AE+MoE"+SUFFIX[k] for k in keys] + \
            ["RBF-CAT+MoE"+SUFFIX[k] for k in keys] + \
            ["VAE-CAT+MoE"+SUFFIX[k] for k in keys] + \
            ["DEC-CAT+MoE"+SUFFIX[k] for k in keys]

res = [load(pJoin(REDUC_SAVE_PATH, k+".pbz2")) for k in m_keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace("-CAT","").replace("+MoE","")
        k2 = m_keys[j].replace("-CAT","").replace("+MoE","")

        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["FC_Frozen"] = calc_pvals(res[2],res[4],res[6])
null_comp_df["FC_Trained"] = calc_pvals(res[3],res[5],res[7])
null_comp_df["Frozen"] = calc_pvals(res[0],res[2],res[4],res[6])
null_comp_df["Trained"] = calc_pvals(res[1],res[3],res[5],res[7])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/reduc/fs_all.csv")

In [129]:
file_name = "reduc_RAW_F_T"
keys = list(comp_df_ae.keys())
x = ["AE_" + k for k in keys for i in range(4)] + \
    ["RBF-CAT_" + k for k in keys for i in range(4)] + \
    ["VAE-CAT_" + k for k in keys for i in range(4)] + \
    ["DEC-CAT_" + k for k in keys for i in range(4)]

y = []
for k in keys:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*4)
fig = plot_bar(results=df_formatted, title="Transferring quality of FS", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Reference perf.", annotation_position="top right")
save_plot(fig, "img/reduc/"+file_name+".svg")
fig.show()


In [130]:
for k in keys:
    for metric in METRICS:
        comp_df_ae[k][metric]= (comp_df_ae[k][metric][0][0], np.round(comp_df_ae[k][metric][1][0], 3))
        comp_df_rbf[k][metric]= (comp_df_rbf[k][metric][0][0], np.round(comp_df_rbf[k][metric][1][0], 3))
        comp_df_vae[k][metric]= (comp_df_vae[k][metric][0][0], np.round(comp_df_vae[k][metric][1][0], 3))
        comp_df_dec[k][metric]= (comp_df_dec[k][metric][0][0], np.round(comp_df_dec[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/reduc/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/reduc/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/reduc/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/reduc/"+file_name+"_dec.csv")

                     Frozen          Trained
recon_error    (66.92, 0.0)     (65.98, 0.0)
adv_error    (99.18, 0.004)  (101.85, 0.599)
rot_error     (104.89, 0.0)    (102.32, 0.0)
delta_rot    (99.23, 0.299)  (100.32, 0.095)
                      Frozen          Trained
recon_error     (40.14, 0.0)     (60.99, 0.0)
adv_error       (91.71, 0.0)   (98.05, 0.513)
rot_error       (93.07, 0.0)    (109.94, 0.0)
delta_rot    (105.67, 0.886)  (101.37, 0.889)
                   Frozen         Trained
recon_error  (46.47, 0.0)    (69.07, 0.0)
adv_error    (96.42, 0.0)  (98.63, 0.015)
rot_error    (85.98, 0.0)   (108.32, 0.0)
delta_rot    (51.79, 0.0)  (90.84, 0.663)
                      Frozen         Trained
recon_error     (62.11, 0.0)   (105.22, 0.0)
adv_error       (86.93, 0.0)  (93.54, 0.831)
rot_error      (110.41, 0.0)   (115.28, 0.0)
delta_rot    (154.89, 0.896)  (97.48, 0.042)


# Summary

In [139]:
comp_df_ae = {"AE_{FE}":{}, "AE_{FS}":{}}
comp_df_rbf = {"RBF_{FC}":{}, "RBF_{FS}":{}}
comp_df_vae = {"VAE_{FC}":{}, "VAE_{FS}":{}}
comp_df_dec = {"DEC_{FC}":{}, "DEC_{FS}":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = "_T"
        f1 = load(pJoin(REF_SAVE_PATH, "AE+MoE.pbz2"))
        path = REDUC_SAVE_PATH if "_{FS}" in k else TRANS_SAVE_PATH
        if base == "AE":
            f2 = load(pJoin(path,base+"+MoE"+suffix+".pbz2"))
        else:
            f2 = load(pJoin(path,base+"-CAT+MoE"+suffix+".pbz2"))
                
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [137]:
null_comp_df = {}
bases = ["AE", "RBF", "VAE", "DEC"] 
keys =    ["AE+MoE_T","RBF-CAT+MoE_T","VAE-CAT+MoE_T","DEC-CAT+MoE_T"]
m_keys =    [
            "AE+MoE_FE", "RBF-CAT+MoE_FC", "VAE-CAT+MoE_FC", "DEC-CAT+MoE_FC",
            "AE+MoE_FS", "RBF-CAT+MoE_FS", "VAE-CAT+MoE_FS", "DEC-CAT+MoE_FS"
            ]

res = [load(pJoin(TRANS_SAVE_PATH, k+".pbz2")) for k in keys] + [load(pJoin(REDUC_SAVE_PATH, k+".pbz2")) for k in keys]

for k in range(len(m_keys)):
    for j in range(k+1,len(m_keys)):
        pvals = calc_pvals(res[k],res[j])
        k1 = m_keys[k].replace("-CAT","").replace("+MoE","")
        k2 = m_keys[j].replace("-CAT","").replace("+MoE","")
        null_comp_df[k1+"_vs_"+k2] = pvals

null_comp_df["FE_vs_FC"] = calc_pvals(res[0],res[1],res[3],res[3])
null_comp_df["FE_vs_FS"] = calc_pvals(res[0],res[5],res[6],res[7])
null_comp_df["FC_vs_FS"] = calc_pvals(res[4],res[5],res[6],res[7])
null_comp_df["All"] = calc_pvals(*res) 
df = pd.DataFrame(null_comp_df).T
df.to_csv("tables/summary_all.csv")

In [140]:
file_name = "Summary"
keys_ae = list(comp_df_ae.keys())
keys_rbf = list(comp_df_rbf.keys())
keys_vae = list(comp_df_vae.keys())
keys_dec = list(comp_df_dec.keys())

x = [k for k in keys_ae for i in range(4)] + \
    [k for k in keys_rbf for i in range(4)] + \
    [k for k in keys_vae for i in range(4)] + \
    [k for k in keys_dec for i in range(4)]

y = []
for k in keys_ae:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys_rbf:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys_vae:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys_dec:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys_ae)*4)
fig = plot_bar(results=df_formatted, title="Summary - Transferred models compared to the reference perf. of AE+MoE", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Reference perf. of AE+MoE", annotation_position="top right")
save_plot(fig, "img/"+file_name+".svg")
fig.show()


In [141]:
for k in keys_ae:
    for metric in METRICS:
        comp_df_ae[k][metric]= (comp_df_ae[k][metric][0][0], np.round(comp_df_ae[k][metric][1][0], 3))
for k in keys_rbf:
    for metric in METRICS:
        comp_df_rbf[k][metric]= (comp_df_rbf[k][metric][0][0], np.round(comp_df_rbf[k][metric][1][0], 3))
for k in keys_vae:
    for metric in METRICS:
        comp_df_vae[k][metric]= (comp_df_vae[k][metric][0][0], np.round(comp_df_vae[k][metric][1][0], 3))
for k in keys_dec:
    for metric in METRICS:
        comp_df_dec[k][metric]= (comp_df_dec[k][metric][0][0], np.round(comp_df_dec[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/"+file_name+"_dec.csv")

                     AE_{FE}          AE_{FS}
recon_error     (67.99, 0.0)     (65.98, 0.0)
adv_error      (98.0, 0.161)  (101.85, 0.599)
rot_error     (88.76, 0.001)    (102.32, 0.0)
delta_rot    (105.56, 0.374)  (100.32, 0.095)
                   RBF_{FC}        RBF_{FS}
recon_error    (68.55, 0.0)    (67.48, 0.0)
adv_error    (96.87, 0.437)  (98.51, 0.288)
rot_error      (85.64, 0.0)    (109.2, 0.0)
delta_rot    (99.81, 0.007)  (100.6, 0.448)
                   VAE_{FC}       VAE_{FS}
recon_error    (79.85, 0.0)   (64.68, 0.0)
adv_error    (100.24, 0.28)  (99.36, 0.87)
rot_error      (93.84, 0.0)  (104.37, 0.0)
delta_rot      (86.01, 0.0)   (77.45, 0.0)
                   DEC_{FC}      DEC_{FS}
recon_error    (48.91, 0.0)  (44.09, 0.0)
adv_error      (93.74, 0.0)  (91.85, 0.0)
rot_error      (77.42, 0.0)  (96.52, 0.0)
delta_rot    (92.11, 0.516)  (86.42, 0.0)
